In [1]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

import bus

geton_station_name = 'geton_stataion_name'

### 1. 정류장 추출

In [2]:
def analyze_usage(df):
    df = df[["geton_station_id", "user_count"]].groupby("geton_station_id").sum()
    df.index.name = "geton_station_id"
    df = df.rename(columns={"user_count":"usage"})
    return df

def create_station_df(usage_df, user_df):
    # extract necessary columns
    station_columns = ["geton_station_id", geton_station_name, "geton_station_latitude", "geton_station_longitude"]
    used_station = usage_df[station_columns].drop_duplicates()
    station_id_df = used_station[["geton_station_id"]]
    
    
    # set tourist column
    usage_df = pd.merge(usage_df, user_df, on = "user_id")
    usage_df = usage_df[["geton_station_id", "user_count", "tourist"]]

    
    # counting user in citizen and trouist
    citizen_user_count = pd.DataFrame(analyze_usage(usage_df[usage_df["tourist"] == 0]))
    tourist_user_count = pd.DataFrame(analyze_usage(usage_df[usage_df["tourist"] == 1]))
    
    
    # counting tag in citizen and trouist
    usage_df["user_count"] = 1
    citizen_tag_count = pd.DataFrame(analyze_usage(usage_df[usage_df["tourist"] == 0]))
    tourist_tag_count = pd.DataFrame(analyze_usage(usage_df[usage_df["tourist"] == 1]))

    
    # create df_list
    df_list = [citizen_user_count, tourist_user_count, citizen_tag_count, tourist_tag_count]
    
    
    # renaming columns(include index)
    usage_column_list = ["citizen_user_count", "tourist_user_count", "citizen_tag_count", "tourist_tag_count"]
    for i in range(len(df_list)):
        df_list[i] = df_list[i].reset_index().rename(columns = {"index":"geton_station_id", "usage": usage_column_list[i]})
    
    
    # processing NaN data
    for i in range(len(df_list)):
        df_list[i] = pd.merge(station_id_df, df_list[i], on = "geton_station_id", how="left")
        df_list[i] = df_list[i].fillna(0)
    
    
    # mergeing each result(count - user, tag) df
    for i in range(len(df_list)):
        used_station= pd.merge(used_station, df_list[i], on="geton_station_id", how="left")
    
    
    # set total count columns
    used_station["total_user_count"] = used_station["citizen_user_count"] + used_station["tourist_user_count"]
    used_station["total_tag_count"] = used_station["citizen_tag_count"] + used_station["tourist_tag_count"]
    
    
    # processing missing values
#     print(used_station.count())
    used_station = used_station.dropna(axis = 0);
    
    
    # renaming columns
    used_station = used_station[(used_station["geton_station_longitude"] != "nan") & (used_station["geton_station_latitude"] != "nan")]
    used_station = used_station.sort_values(by="geton_station_id", ascending = False)
    used_station = used_station.rename(columns = {"geton_station_id" :'station_id',
                             geton_station_name: "station_name", 
                             "geton_station_latitude":"station_y",
                             "geton_station_longitude":"station_x",})
    
    return used_station

### 2. 주소 추가

In [3]:
import xml.etree.ElementTree as elemTree
import requests

def set_address_column(df):
    df = pd.DataFrame.copy(df)
    df["station_address"] = ""
    for i in tqdm(df.index):
        df.loc[i, "station_address"] = get_address(float(df.loc[i, "station_x"]), float(df.loc[i, "station_y"]))
    return df

def get_address(loc_x, loc_y, min_x = 126.531891, min_y = 33.399409, key = "E20F6493-C13D-3F6F-AC90-D5BB2F239901"):
    loc_x = round(float(loc_x), 7)
    loc_y = round(float(loc_y), 7)
    url_form = "http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={},{}&format=xml&type=both&zipcode=true&simple=false&key={}"
    url = url_form.format(loc_x, loc_y, key)
    response = requests.get(url)
    tree = elemTree.fromstring(response.text)
    branch = ""
    try:
#     road = tree.find("result").find("item[2]").find("text").text
        branch = tree.find("result").find("item[1]").find("text").text
    except:
#         print(loc_x, loc_y)
        x_left = str(int(loc_x))
        y_left = str(int(loc_y))
        
        
        if min_x < loc_x:
            x_right = str((int(loc_x*100000)-2)%100000)
        else:
            x_right = str((int(loc_x*100000)+2)%100000)
            
        if min_y < loc_y:
            y_right = str((int(loc_y*100000)-2)%100000)
        else:
            y_right = str((int(loc_y*100000)+2)%100000)
            
        loc_x = float(x_left+"."+x_right)
        loc_y = float(y_left+"."+y_right)
        return get_address(loc_x, loc_y)
            
    return branch

### main

In [4]:
# setting clasification period(start, end)
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

# load usage data
input_path_list = bus.make_input_path(start_date, end_date)
usage_df = bus.load_total_usage_data(input_path_list)
usage_df = bus.preprocessing_missing_data_from_usage_df(usage_df)

# load user_data
user_df = bus.create_user_df()

# get station data
station_df = create_station_df(usage_df, user_df)
print("추출된 정류장 개수: ",len(station_df))

# setting address column
station_df = set_address_column(station_df)

# sorting
station_df = station_df.sort_values(by="station_id")

# storing
station_df.to_csv("station_list.csv", encoding="CP949", index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [04:24<00:00,  2.97s/it]
D:\Users\Embedded\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
  0%|                                                                                 | 1/3952 [00:00<07:52,  8.35it/s]

추출된 정류장 개수:  3952


100%|██████████████████████████████████████████████████████████████████████████████| 3952/3952 [05:01<00:00, 13.11it/s]


### 결과

In [5]:
station_df

,station_id,station_name,station_y,station_x,citizen_user_count,tourist_user_count,citizen_tag_count,tourist_tag_count,total_user_count,total_tag_count,station_address
456,1,국제여객선터미널,33.52438,126.54433,4003.0,2122.0,3912.0,1875.0,6125.0,5787.0,제주특별자치도 제주시 건입동 908-20
638,2,오광로입구,33.49527,126.45618,8173.0,2550.0,7897.0,2418.0,10723.0,10315.0,제주특별자치도 제주시 이호이동 1587-4
2093,3,오광로입구,33.49546,126.45623,917.0,311.0,890.0,294.0,1228.0,1184.0,제주특별자치도 제주시 이호이동 1587-1
537,4,이호2동,33.49488,126.46137,9868.0,230.0,9804.0,211.0,10098.0,10015.0,제주특별자치도 제주시 이호이동 803-6
1802,5,이호2동,33.49508,126.46122,2992.0,128.0,2964.0,124.0,3120.0,3088.0,제주특별자치도 제주시 이호이동 803-5
...,...,...,...,...,...,...,...,...,...,...,...
3528,6115048,우도봉입구,33.49558,126.95411,22.0,2.0,21.0,2.0,24.0,23.0,제주특별자치도 제주시 우도면 연평리 1666-2
1824,6115052,비양동,33.51123,126.96567,1780.0,433.0,1567.0,393.0,2213.0,1960.0,제주특별자치도 제주시 우도면 연평리 133-1
1929,6115059,하우목동,33.50731,126.95093,305.0,181.0,245.0,163.0,486.0,408.0,제주특별자치도 제주시 우도면 연평리 2164-2
1317,6115100,서귀포시외버스터미널,33.24873,126.50799,3360.0,343.0,3324.0,333.0,3703.0,3657.0,제주특별자치도 서귀포시 법환동 843
